In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/train_data.csv')

In [3]:
train_data.shape

(12379, 31)

In [4]:
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


In [5]:
columns = train_data.dtypes.reset_index()
num_cols = columns[columns[0]=='int64']['index'].values
from sklearn.preprocessing import MaxAbsScaler
s = MaxAbsScaler()
train_data[num_cols] = s.fit_transform(train_data[num_cols])
#train_data[num_cols] = train_data[num_cols].astype('int')

In [6]:
train_data = train_data.drop(['car'],axis=1)
train_data = train_data.drop(['restuarant_opposite_direction_house'],axis=1)


import category_encoders as ce
encoder= ce.OrdinalEncoder(cols=['offer expiration'],return_df=True,
                           mapping=[{'col':'offer expiration','mapping':{'10hours':1,'2days':2}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['no_visited_Cold drinks'],return_df=True,
                           mapping=[{'col':'no_visited_Cold drinks','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['Restaur_spend_less_than20'],return_df=True,
                           mapping=[{'col':'Restaur_spend_less_than20','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['Marital Status'],return_df=True,
                           mapping=[{'col':'Marital Status','mapping':{'Married partner':0,'Single':3,'Divorced':2,'Unmarried partner':4,'Widowed':1}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['gender'],return_df=True,
                           mapping=[{'col':'gender','mapping':{'Female':0,'Male':1}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['Customer type'],return_df=True,
                           mapping=[{'col':'Customer type','mapping':{'Individual':0,'With Family':1,'With Kids':2,'With Colleagues':3}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['no_Take-aways'],return_df=True,
                           mapping=[{'col':'no_Take-aways','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
train_data = encoder.fit_transform(train_data)

train_data = train_data.drop(['Restaur_spend_greater_than20'],axis=1)

encoder= ce.OrdinalEncoder(cols=['Climate'],return_df=True,
                           mapping=[{'col':'Climate','mapping':{'Spring':0,'Summer':1,'Winter':2}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['drop location'],return_df=True,
                           mapping=[{'col':'drop location','mapping':{'Location A':0,'Location B':1,'Location c':2}}])
train_data = encoder.fit_transform(train_data)

encoder= ce.OrdinalEncoder(cols=['no_visited_bars'],return_df=True,
                           mapping=[{'col':'no_visited_bars','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
train_data = encoder.fit_transform(train_data)

In [7]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

train_data['income_range'] = label_encoder.fit_transform(train_data['income_range'])
train_data['restaurant type'] = label_encoder.fit_transform(train_data['restaurant type'])
train_data['age'] = label_encoder.fit_transform(train_data['age'])
train_data['Qualification'] = label_encoder.fit_transform(train_data['Qualification'])

In [8]:
train_data['drop location'] = train_data['drop location'].astype('int')
train_data['no_Take-aways'] = train_data['no_Take-aways'].astype('int')
train_data['Restaur_spend_less_than20'] = train_data['Restaur_spend_less_than20'].astype('int')
train_data['no_visited_Cold drinks'] = train_data['no_visited_Cold drinks'].astype('int')
train_data['no_visited_bars'] = train_data['no_visited_bars'].astype('int')

In [9]:
columns = train_data.dtypes.reset_index()
selected_columns = columns[columns[0]=='int64']['index'].values
selected_columns

array(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'Restaur_spend_less_than20', 'Marital Status', 'restaurant type',
       'age', 'no_visited_bars', 'gender', 'Customer type',
       'Qualification', 'no_Take-aways', 'Climate', 'drop location'],
      dtype=object)

In [10]:
train_data[selected_columns] = train_data[selected_columns].astype('float')


In [11]:
columns = train_data.dtypes.reset_index()
selected_columns = columns[columns[0]=='float64']['index'].values
selected_columns

array(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer',
       'Restaur_spend_less_than20', 'Marital Status', 'restaurant type',
       'age', 'Prefer western over chinese',
       'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly',
       'Customer type', 'Qualification', 'is foodie', 'no_Take-aways',
       'has Children', 'visit restaurant with rating (avg)',
       'temperature', 'Travel Time', 'Climate', 'drop location',
       'Prefer home food'], dtype=object)

In [12]:
X_train = train_data[selected_columns]
y_train = train_data['Offer Accepted']

In [13]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [14]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=30)

In [15]:
clf.fit(X_train,y_train)

AdaBoostClassifier(n_estimators=30)

In [16]:
test_data = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')

In [17]:
colums = test_data.dtypes.reset_index()
num_coles = colums[colums[0]=='int64']['index'].values
from sklearn.preprocessing import MaxAbsScaler
ss = MaxAbsScaler()
test_data[num_coles] = ss.fit_transform(test_data[num_coles])
#test_data[num_coles] = test_data[num_coles].astype('int')

In [18]:
import category_encoders as ce
encoder= ce.OrdinalEncoder(cols=['offer expiration'],return_df=True,
                           mapping=[{'col':'offer expiration','mapping':{'10hours':1,'2days':2}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['no_visited_Cold drinks'],return_df=True,
                           mapping=[{'col':'no_visited_Cold drinks','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['Restaur_spend_less_than20'],return_df=True,
                           mapping=[{'col':'Restaur_spend_less_than20','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['Marital Status'],return_df=True,
                           mapping=[{'col':'Marital Status','mapping':{'Married partner':0,'Single':3,'Divorced':2,'Unmarried partner':4,'Widowed':1}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['gender'],return_df=True,
                           mapping=[{'col':'gender','mapping':{'Female':0,'Male':1}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['Customer type'],return_df=True,
                           mapping=[{'col':'Customer type','mapping':{'Individual':0,'With Family':1,'With Kids':2,'With Colleagues':3}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['no_Take-aways'],return_df=True,
                           mapping=[{'col':'no_Take-aways','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
test_data = encoder.fit_transform(test_data)

test_data = test_data.drop(['Restaur_spend_greater_than20'],axis=1)

encoder= ce.OrdinalEncoder(cols=['Climate'],return_df=True,
                           mapping=[{'col':'Climate','mapping':{'Spring':0,'Summer':1,'Winter':2}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['drop location'],return_df=True,
                           mapping=[{'col':'drop location','mapping':{'Location A':0,'Location B':1,'Location c':2}}])
test_data = encoder.fit_transform(test_data)

encoder= ce.OrdinalEncoder(cols=['no_visited_bars'],return_df=True,
                           mapping=[{'col':'no_visited_bars','mapping':{'4~8':4,'less1':2,'never':1,'1~3':3,'gt8':5}}])
test_data = encoder.fit_transform(test_data)

In [19]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

test_data['income_range'] = label_encoder.fit_transform(test_data['income_range'])
test_data['restaurant type'] = label_encoder.fit_transform(test_data['restaurant type'])
test_data['age'] = label_encoder.fit_transform(test_data['age'])
test_data['Qualification'] = label_encoder.fit_transform(test_data['Qualification'])
test_data['drop location'] = test_data['drop location'].astype('int')
test_data['no_Take-aways'] = test_data['no_Take-aways'].astype('int')
test_data['Restaur_spend_less_than20'] = test_data['Restaur_spend_less_than20'].astype('int')
test_data['no_visited_Cold drinks'] = test_data['no_visited_Cold drinks'].astype('int')
test_data['no_visited_bars'] = test_data['no_visited_bars'].astype('int')

In [20]:
train_data[selected_columns] = train_data[selected_columns].astype('float')

In [21]:
columns = train_data.dtypes.reset_index()
selected_columns = columns[columns[0]=='float64']['index'].values
selected_columns

array(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer',
       'Restaur_spend_less_than20', 'Marital Status', 'restaurant type',
       'age', 'Prefer western over chinese',
       'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly',
       'Customer type', 'Qualification', 'is foodie', 'no_Take-aways',
       'has Children', 'visit restaurant with rating (avg)',
       'temperature', 'Travel Time', 'Climate', 'drop location',
       'Prefer home food'], dtype=object)

In [22]:
X_test = test_data[selected_columns]

In [23]:
pred = clf.predict(X_test)

In [24]:
arr = []
for elem in pred :
    if elem==1:
        arr.append('Yes')
    else:
        arr.append('No')

In [25]:
submission = pd.DataFrame(columns=['Id','Offer Accepted'])
submission['Id'] = [i for i in range(len(pred))]

submission['Offer Accepted'] = arr
submission.head()

,Id,Offer Accepted
0,0,Yes
1,1,Yes
2,2,No
3,3,No
4,4,Yes


In [26]:
submission.to_csv('submission.csv',index=False)